<a href="https://colab.research.google.com/github/hyunholee26/Tutorial-Hydrology-data-collection-using-WAMIS-OpenAPI/blob/main/Tutorial_Hydrology_data_collection_using_WAMIS_OpenAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Tutorial] WAMIS OpenAPI를 이용한 수문관측데이터 수집

## 1. 국가수자원관리종합정보시스템(WAMIS)
 - 환경부 산하 한강홍수통제소에서 관리중인 시스템으로 환경부, 기상청, 한국수자원공사, 한국농어촌공사 등을 통해 수집된 **강수량, 수위, 기상 등 수문데이터를 통합하여 OpenAPI 방식으로 제공**
   - URL : [http://www.wamis.go.kr/](http://www.wamis.go.kr/)
   - 본 튜토리얼에서는 WAMIS에서 제공하는 OpenAPI를 이용함에 따라, 미리 **WAMIS의 OpenAPI 인증키를 발급받아야 합니다.**

```
[OpenAPI 발급절차]
1. [http://www.wamis.go.kr/](http://www.wamis.go.kr/) 접속
2. OpenAPI 메뉴 클릭
3. 로그인
4. OpenAPI활용 신청 및 키 확인
```

## 2. OpenAPI
  - OpenAPI란 S/W개발에 사용가능한 **공개된 API를 의미**하며, 본 튜토리얼에서는 WAMIS에서 제공하는 OpenAPI를 이용하여 수문데이터를 수집하는 예제를 제공하고자 합니다.

## 3. Tutoral 주요 내용
  - WAMIS OpenAPI 호출하여 단기간(6개월 미만) 시단위 강우데이터 수집하기
  - 반복적으로 OpenAPI를 호출하여 장기간(6개월 이상) 시단위 강우데이터 수집하기


## Step 1. WAMIS OpenAPI 호출하여 단기간(6개월 미만) 시단위 강우데이터 수집하기



In [1]:
# 관련 라이브러리 import하기
import pandas as pd
import numpy as np

from urllib.request import Request, urlopen, build_opener, HTTPRedirectHandler, addinfourl
from urllib.parse import urlencode, quote_plus

from bs4 import BeautifulSoup

import json

import datetime

In [ ]:
# WAMIS OpenAPI 호출을 위한 URL 및 인자 셋팅

# WAMIS 강우량데이터 OpenAPI URL
url = 'http://www.wamis.go.kr:8080/wamis/openapi/wkw/rf_hrdata'

# 인증키
key = # 사용자 키입력

# 관측소코드, 대관령의 강우관측소 코드는 10011100임
obscd = '10011100'

# 시작일
startdt = '20220101'

# 종료일
enddt = '20220131'

# OpenAPI 쿼리
queryParams = '?' + urlencode({ quote_plus('key') : key, quote_plus('obscd') : str(obscd), quote_plus('startdt') : str(startdt), quote_plus('enddt') : str(enddt), quote_plus('output') : 'json' })

In [25]:
# 302에러코드 처리를 위한 handler
class NoRedirectHandler(HTTPRedirectHandler):
    def http_error_302(self, req, fp, code, msg, headers):
        infourl = addinfourl(fp, headers, req.get_full_url())
        infourl.status = code
        infourl.code = code
        print('302 Error exception')
        raise Exception('redirection error')
        return infourl
    http_error_300 = http_error_302
    http_error_301 = http_error_302
    http_error_303 = http_error_302
    http_error_307 = http_error_302

In [26]:
# 강우량 데이터 OpenAPI와 쿼리 구문을 합치고, OpenAPI를 호출하면 json형식으로 결과가 리턴됨
request = Request(url + queryParams)
request.get_method = lambda: 'GET'
opener = build_opener(NoRedirectHandler())    
response_body = opener.open(request).read().decode('utf-8')
response_body

'{"result":{"code":"success","msg":"완료되었습니다!"},"count":744,"list":[{"ymdh":"2022010101","rf":"0"},{"ymdh":"2022010102","rf":"0"},{"ymdh":"2022010103","rf":"0"},{"ymdh":"2022010104","rf":"0"},{"ymdh":"2022010105","rf":"0"},{"ymdh":"2022010106","rf":"0"},{"ymdh":"2022010107","rf":"0"},{"ymdh":"2022010108","rf":"0"},{"ymdh":"2022010109","rf":"0"},{"ymdh":"2022010110","rf":"0"},{"ymdh":"2022010111","rf":"0"},{"ymdh":"2022010112","rf":"0"},{"ymdh":"2022010113","rf":"0"},{"ymdh":"2022010114","rf":"0"},{"ymdh":"2022010115","rf":"0"},{"ymdh":"2022010116","rf":"0"},{"ymdh":"2022010117","rf":"0"},{"ymdh":"2022010118","rf":"0"},{"ymdh":"2022010119","rf":"0"},{"ymdh":"2022010120","rf":"0"},{"ymdh":"2022010121","rf":"0"},{"ymdh":"2022010122","rf":"0"},{"ymdh":"2022010123","rf":"0"},{"ymdh":"2022010124","rf":"0"},{"ymdh":"2022010201","rf":"0"},{"ymdh":"2022010202","rf":"0"},{"ymdh":"2022010203","rf":"0"},{"ymdh":"2022010204","rf":"0"},{"ymdh":"2022010205","rf":"0"},{"ymdh":"2022010206","rf":"0"},{"y

In [28]:
# json형식으로 리턴된 결과를 정리
json_output = json.loads(response_body)
json_output

{'count': 744,
 'list': [{'rf': '0', 'ymdh': '2022010101'},
  {'rf': '0', 'ymdh': '2022010102'},
  {'rf': '0', 'ymdh': '2022010103'},
  {'rf': '0', 'ymdh': '2022010104'},
  {'rf': '0', 'ymdh': '2022010105'},
  {'rf': '0', 'ymdh': '2022010106'},
  {'rf': '0', 'ymdh': '2022010107'},
  {'rf': '0', 'ymdh': '2022010108'},
  {'rf': '0', 'ymdh': '2022010109'},
  {'rf': '0', 'ymdh': '2022010110'},
  {'rf': '0', 'ymdh': '2022010111'},
  {'rf': '0', 'ymdh': '2022010112'},
  {'rf': '0', 'ymdh': '2022010113'},
  {'rf': '0', 'ymdh': '2022010114'},
  {'rf': '0', 'ymdh': '2022010115'},
  {'rf': '0', 'ymdh': '2022010116'},
  {'rf': '0', 'ymdh': '2022010117'},
  {'rf': '0', 'ymdh': '2022010118'},
  {'rf': '0', 'ymdh': '2022010119'},
  {'rf': '0', 'ymdh': '2022010120'},
  {'rf': '0', 'ymdh': '2022010121'},
  {'rf': '0', 'ymdh': '2022010122'},
  {'rf': '0', 'ymdh': '2022010123'},
  {'rf': '0', 'ymdh': '2022010124'},
  {'rf': '0', 'ymdh': '2022010201'},
  {'rf': '0', 'ymdh': '2022010202'},
  {'rf': '0', '

In [30]:
# 최종적으로 pandas dataframe으로 변환, ymdh는 관측 연월일시를, rf는 해당지점의 강우량을 의미함
df_temp = pd.json_normalize(json_output['list'])
df_temp

,ymdh,rf
0,2022010101,0
1,2022010102,0
2,2022010103,0
3,2022010104,0
4,2022010105,0
...,...,...
739,2022013120,0
740,2022013121,0
741,2022013122,0
742,2022013123,0


## Step2. 반복적으로 OpenAPI를 호출하여 장기간(6개월 이상) 시단위 강우데이터 수집하기

In [32]:
# WAMIS에서 제공하는 강우데이터 OpenAPI는 최대 6개월간의 데이터가 조회 가능함에 따라, 6개월단위로 OpenAPI를 호출할 수 있도록 기간을 끊어서 관리

raw_period = {'startdt': ['20140101', '20140701', '20150101', '20150701', '20160101', '20160701', '20170101', '20170701', '20180101', '20180701', '20190101', '20190701', '20200101', '20200701', '20210101'],
                'enddt': ['20140630', '20141231', '20150630', '20151231', '20160630', '20161231', '20170630', '20171231', '20180630', '20181231', '20190630', '20191231', '20200630', '20201231', '20210630']}

df_period = pd.DataFrame(raw_period)
df_period

,startdt,enddt
0,20140101,20140630
1,20140701,20141231
2,20150101,20150630
3,20150701,20151231
4,20160101,20160630
5,20160701,20161231
6,20170101,20170630
7,20170701,20171231
8,20180101,20180630
9,20180701,20181231


In [36]:
# exception 처리 로직을 포함하여 앞의 과정을 하나의 함수로 작성

def get_rf_by_openapi(obscd) :
  url = 'http://www.wamis.go.kr:8080/wamis/openapi/wkw/wl_hrdata'
  key = # 사용자 키입력
 
  # 시간단위 생성
  start = datetime.datetime.strptime("2014010101", "%Y%m%d%H")
  end = datetime.datetime.strptime("2021063023", "%Y%m%d%H")
  date_generated = [start + datetime.timedelta(hours=x) for x in range(0, (end-start).days * 24 + 23)]
  df = pd.DataFrame(date_generated, columns =['Date'])
  df['ymdh'] = df['Date'].apply(lambda x: int(x.strftime('%Y%m%d%H'))) 

  # OpenAPI 호출
  for j, row_j in df_period.iterrows():
    startdt = row_j['startdt'] 
    enddt = row_j['enddt'] 
    queryParams = '?' + urlencode({ quote_plus('key') : key, quote_plus('obscd') : str(obscd), quote_plus('startdt') : str(startdt), quote_plus('enddt') : str(enddt), quote_plus('output') : 'json' })
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    opener = build_opener(NoRedirectHandler())
    
    trycount = 10 # retry counter 
    while trycount > 0 :
      try:
        response_body = opener.open(request).read().decode('utf-8')
        break;
      except:
        trycount = trycount - 1
        continue;

      if trycount > 0:
        jj = json.loads(response_body)
        df_temp = pd.json_normalize(jj['list'])
        df = df.merge(temp, how='left', on = 'ymdh')
  return (df)

df_temp = get_rf_by_openapi('10011100')
        

302 Error exception


In [37]:
# 2014년 부터 2021년 6월까지 강우데이터 수집 결과 확인
df_temp

,Date,ymdh
0,2014-01-01 01:00:00,2014010101
1,2014-01-01 02:00:00,2014010102
2,2014-01-01 03:00:00,2014010103
3,2014-01-01 04:00:00,2014010104
4,2014-01-01 05:00:00,2014010105
...,...,...
65706,2021-06-30 19:00:00,2021063019
65707,2021-06-30 20:00:00,2021063020
65708,2021-06-30 21:00:00,2021063021
65709,2021-06-30 22:00:00,2021063022
